In [1]:
pip install regex matplotlib sastrawi xlsxwriter nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 17.1 MB/s eta 0:00:00


In [3]:
#import modul
import pandas as pd
import numpy as np
import re as reg
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
data = pd.read_csv('Kasus1 (2).csv', delimiter=';')
data

,screen_name,username,user_id,tweet_id,tweet_url,timestamp,timestamp_epochs,text,mengandung pelecehan,text_html,...,has_media,img_urls,video_url,likes,retweets,replies,is_replied,is_reply_to,parent_tweet_id,reply_to_users
0,BimaBurhanuddin,Burhanuddin Bima,3058961791,"1,12E+18",/BimaBurhanuddin/status/1124100611925000193,02/05/2019 23:57,1556841445,Apa yg bisa diharapkan dari politisi gaya pela...,1,"<p class=""TweetTextSize js-tweet-text tweet-te...",...,False,[],NaN,0,0,0,False,False,NaN,[]
1,ngowatchindo,Waw Lam'alif,"8,95E+17","1,12E+18",/ngowatchindo/status/1124099972046163968,02/05/2019 23:54,1556841292,"Kalau sy jadi ⁦@DennyJA_WORLD⁩ , Syahganda aka...",1,"<p class=""TweetTextSize js-tweet-text tweet-te...",...,False,[],NaN,1,1,0,False,False,NaN,[]
2,socmed_anak,AnakSocmed,174945628,"1,12E+18",/socmed_anak/status/1124098250083131393,02/05/2019 23:48,1556840882,Di @PDemokrat banyak org2 yg punya akal sehat ...,1,"<p class=""TweetTextSize js-tweet-text tweet-te...",...,False,[],NaN,0,0,0,False,False,NaN,[]
3,PelajaranOnline,PelajaranOnline,589466301,"1,12E+18",/PelajaranOnline/status/1124087714154975232,02/05/2019 23:06,1556838370,Cypridophobia adalah fobia atau takut pada pel...,0,"<p class=""TweetTextSize js-tweet-text tweet-te...",...,False,[],NaN,0,0,0,False,False,NaN,[]
4,Lee_Botak,Lee Botak,509046818,"1,12E+18",/Lee_Botak/status/1124084310640869379,02/05/2019 22:52,1556837558,Yang RAMAI kerja waktu malam ni biasanya PELAC...,1,"<p class=""TweetTextSize js-tweet-text tweet-te...",...,False,[],NaN,0,0,0,False,False,NaN,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,OposisiRasional,Oposisi_Rasional,"1,13E+18","1,17E+18",/OposisiRasional/status/1174274919196188672,18/09/2019 10:52,1568803932,Letakan DiKepalak Sepatunya Klo Tidak Mau Koto...,0,"<p class=""TweetTextSize js-tweet-text tweet-te...",...,False,[],NaN,56,6,1,True,True,"1,17E+18","[{'screen_name': 'LisaAmartatara3', 'user_id':..."
396,Adnity05,Adnity,"1,11E+18","1,17E+18",/Adnity05/status/1174270298792898560,18/09/2019 10:33,1568802830,Media pelacur lahir dari rahim yg tak jujur. h...,1,"<p class=""TweetTextSize js-tweet-text tweet-te...",...,False,[],NaN,0,0,0,False,False,NaN,[]
397,adebaihaki1,Bakayarou,"1,09E+18","1,22E+18",/adebaihaki1/status/1215058126501580802,08/01/2020 23:50,1578527406,Jadi pelacur ko nanti dijawa nak. Jangan...,0,"<p class=""TweetTextSize js-tweet-text tweet-te...",...,False,[],NaN,0,0,1,True,True,"1,21E+18","[{'screen_name': 'mhdaziz_', 'user_id': '99137..."
398,RICHARDSAMBORA3,RICHARD SAMBORA,"1,16E+18","1,17E+18",/RICHARDSAMBORA3/status/1174272471991799808,18/09/2019 10:42,1568803348,Kok jadi pelacur,1,"<p class=""TweetTextSize js-tweet-text tweet-te...",...,False,[],NaN,0,0,0,False,True,"1,17E+18","[{'screen_name': 'Openbo15989801', 'user_id': ..."


In [5]:
#Preprocessing
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

slangs={'yg':'yang', 'tdk':'tidak', 'pd':'pada', 'mlh':'malah', 'jgn':'jangan', 'jg':'juga', 'tp':'tapi', 'blkg': 'belakang', 'dr':'dari', 'klo':'kalo', 'lg':'lagi', 'btw':'buat',
        'dlm':'dalam','dgn':'dengan', 'poto':'foto', 'g':'tidak', 'n':'dan', 'ad':'ada', 'brp': 'berapa', "abis": "habis", "ad": "ada", "adlh": "adalah", "afaik": "as far as i know",
        "ahaha": "haha", "aj": "saja", "ajep-ajep": "dunia gemerlap", "ak": "saya", "akika": "aku", "akkoh": "aku", "akuwh": "aku", "alay": "norak", "alow": "halo", "ambilin": "ambilkan",
        "ancur": "hancur", "anjrit": "anjing", "anter": "antar", "ap2": "apa-apa", "apasih": "apa sih", "apes": "sial", "aps": "apa", "aq": "saya", "aquwh": "aku", "asbun": "asal bunyi",
        "aseekk": "asyik", "asekk": "asyik", "asem": "asam", "aspal": "asli tetapi palsu", "astul": "asal tulis", "ato": "atau", "au ah": "tidak mau tahu", "awak": "saya", "ay": "sayang",
        "ayank": "sayang", "b4": "sebelum", "bakalan": "akan", "bandes": "bantuan desa", "bangedh": "banget", "banpol": "bantuan polisi", "banpur": "bantuan tempur", "basbang": "basi",
        "bcanda": "bercanda", "bdg": "bandung", "begajulan": "nakal", "beliin": "belikan", "bencong": "banci", "bentar": "sebentar", "ber3": "bertiga", "beresin": "membereskan", "bete":
        "bosan", "beud": "banget", "bg": "abang", "bgmn": "bagaimana", "bgt": "banget", "bijimane": "bagaimana", "bintal": "bimbingan mental", "bkl": "akan", "bknnya": "bukannya",
        "blegug": "bodoh", "blh": "boleh", "bln": "bulan", "blum": "belum", "bnci": "benci", "bnran": "yang benar", "bodor": "lucu", "bokap": "ayah", "boker": "buang air besar", "bokis":
        "bohong", "boljug": "boleh juga", "bonek": "bocah nekat", "boyeh": "boleh", "br": "baru", "brg": "bareng", "bro": "saudara laki-laki", "bru": "baru", "bs": "bisa", "bsen": "bosan",
        "bt": "buat", "btw": "ngomong-ngomong", "buaya": "tidak setia", "bubbu": "tidur", "bubu": "tidur", "bumil": "ibu hamil", "bw": "bawa", "bwt": "buat", "byk": "banyak", "byrin": "bayarkan",
        "cabal": "sabar", "cadas": "keren", "calo": "makelar", "can": "belum", "capcus": "pergi", "caper": "cari perhatian", "ce": "cewek", "cekal": "cegah tangkal", "cemen": "penakut",
        "cengengesan": "tertawa", "cepet": "cepat", "cew": "cewek", "chuyunk": "sayang", "cimeng": "ganja", "cipika cipiki": "cium pipi kanan cium pipi kiri", "ciyh": "sih", "ckepp": "cakep",
        "ckp": "cakep", "cmiiw": "correct me if i'm wrong", "cmpur": "campur", "cong": "banci", "conlok": "cinta lokasi", "cowwyy": "maaf", "cp": "siapa", "cpe": "capek", "cppe": "capek",
        "cucok": "cocok", "cuex": "cuek", "cumi": "Cuma miscall", "cups": "culun", "curanmor": "pencurian kendaraan bermotor", "curcol": "curahan hati colongan", "cwek": "cewek", "cyin": "cinta",
        "d": "di", "dah": "deh", "dapet": "dapat", "de": "adik", "dek": "adik", "demen": "suka", "deyh": "deh", "dgn": "dengan", "diancurin": "dihancurkan", "dimaafin": "dimaafkan",
        "dimintak": "diminta", "disono": "di sana", "dket": "dekat", "dkk": "dan kawan-kawan", "dll": "dan lain-lain", "dlu": "dulu", "dngn": "dengan", "dodol": "bodoh", "doku": "uang",
        "dongs": "dong", "dpt": "dapat", "dri": "dari", "drmn": "darimana", "drtd": "dari tadi", "dst": "dan seterusnya", "dtg": "datang", "duh": "aduh", "duren": "durian", "ed": "edisi",
        "egp": "emang gue pikirin", "eke": "aku", "elu": "kamu", "emangnya": "memangnya", "emng": "memang", "endak": "tidak", "enggak": "tidak", "envy": "iri", "ex": "mantan", "fax": "facsimile",
        "fifo": "first in first out", "folbek": "follow back", "fyi": "sebagai informasi", "gaada": "tidak ada uang", "gag": "tidak", "gaje": "tidak jelas", "gak papa": "tidak apa-apa",
        "gan": "juragan", "gaptek": "gagap teknologi", "gatek": "gagap teknologi", "gawe": "kerja", "gbs": "tidak bisa", "gebetan": "orang yang disuka", "geje": "tidak jelas",
        "gepeng": "gelandangan dan pengemis", "ghiy": "lagi", "gile": "gila", "gimana": "bagaimana", "gino": "gigi nongol", "githu": "gitu", "gj": "tidak jelas", "gmana": "bagaimana",
        "gn": "begini", "goblok": "bodoh", "golput": "golongan putih", "gowes": "mengayuh sepeda", "gpny": "tidak punya", "gr": "gede rasa", "gretongan": "gratisan", "gtau": "tidak tahu",
        "gua": "saya", "guoblok": "goblok", "gw": "saya", "ha": "tertawa", "haha": "tertawa", "hallow": "halo", "hankam": "pertahanan dan keamanan", "hehe": "he", "helo": "halo", "hey": "hai",
        "hlm": "halaman", "hny": "hanya", "hoax": "isu bohong", "hr": "hari", "hrus": "harus", "hubdar": "perhubungan darat", "huff": "mengeluh", "hum": "rumah", "humz": "rumah", "ilang": "hilang",
        "ilfil": "tidak suka", "imho": "in my humble opinion", "imoetz": "imut", "item": "hitam", "itungan": "hitungan", "iye": "iya", "ja": "saja", "jadiin": "jadi", "jaim": "jaga image",
        "jayus": "tidak lucu", "jdi": "jadi", "jem": "jam", "jga": "juga", "jgnkan": "jangankan", "jir": "anjing", "jln": "jalan", "jomblo": "tidak punya pacar", "jubir": "juru bicara",
        "jutek": "galak", "k": "ke", "kab": "kabupaten", "kabor": "kabur", "kacrut": "kacau", "kadiv": "kepala divisi", "kagak": "tidak", "kalo": "kalau", "kampret": "sialan",
        "kamtibmas": "keamanan dan ketertiban masyarakat", "kamuwh": "kamu", "kanwil": "kantor wilayah", "karna": "karena", "kasubbag": "kepala subbagian", "katrok": "kampungan",
        "kayanya": "kayaknya", "kbr": "kabar", "kdu": "harus", "kec": "kecamatan", "kejurnas": "kejuaraan nasional", "kekeuh": "keras kepala", "kel": "kelurahan", "kemaren": "kemarin",
        "kepengen": "mau", "kepingin": "mau", "kepsek": "kepala sekolah", "kesbang": "kesatuan bangsa", "kesra": "kesejahteraan rakyat", "ketrima": "diterima", "kgiatan": "kegiatan",
        "kibul": "bohong", "kimpoi": "kawin", "kl": "kalau", "klianz": "kalian", "kloter": "kelompok terbang", "klw": "kalau", "km": "kamu", "kmps": "kampus", "kmrn": "kemarin", "knal": "kenal",
        "knp": "kenapa", "kodya": "kota madya", "komdis": "komisi disiplin", "komsov": "komunis sovyet", "kongkow": "kumpul bareng teman-teman", "kopdar": "kopi darat", "korup": "korupsi",
        "kpn": "kapan", "krenz": "keren", "krm": "kirim", "kt": "kita", "ktmu": "ketemu", "ktr": "kantor", "kuper": "kurang pergaulan", "kw": "imitasi", "kyk": "seperti", "la": "lah",
        "lam": "salam", "lamp": "lampiran", "lanud": "landasan udara", "latgab": "latihan gabungan", "lebay": "berlebihan", "leh": "boleh", "lelet": "lambat", "lemot": "lambat", "lgi": "lagi",
        "lgsg": "langsung", "liat": "lihat", "litbang": "penelitian dan pengembangan", "lmyn": "lumayan", "lo": "kamu", "loe": "kamu", "lola": "lambat berfikir", "louph": "cinta", "low": "kalau",
        "lp": "lupa", "luber": "langsung, umum, bebas, dan rahasia", "luchuw": "lucu", "lum": "belum", "luthu": "lucu", "lwn": "lawan", "maacih": "terima kasih", "mabal": "bolos", "macem": "macam", "macih": "masih", "maem": "makan", "magabut": "makan gaji buta", "maho": "homo", "mak jang": "kaget", "maksain": "memaksa", "malem": "malam", "mam": "makan", "maneh": "kamu", "maniez": "manis", "mao": "mau", "masukin": "masukkan", "melu": "ikut", "mepet": "dekat sekali", "mgu": "minggu", "migas": "minyak dan gas bumi", "mikol": "minuman beralkohol", "miras": "minuman keras", "mlah": "malah", "mngkn": "mungkin", "mo": "mau", "mokad": "mati", "moso": "masa", "mpe": "sampai", "msk": "masuk", "mslh": "masalah", "mt": "makan teman", "mubes": "musyawarah besar", "mulu": "melulu", "mumpung": "selagi", "munas": "musyawarah nasional", "muntaber": "muntah dan berak", "musti": "mesti", "muupz": "maaf", "mw": "now watching", "n": "dan", "nanam": "menanam", "nanya": "bertanya", "napa": "kenapa", "napi": "narapidana", "napza": "narkotika, alkohol, psikotropika, dan zat adiktif ", "narkoba": "narkotika, psikotropika, dan obat terlarang", "nasgor": "nasi goreng", "nda": "tidak", "ndiri": "sendiri", "ne": "ini", "nekolin": "neokolonialisme", "nembak": "menyatakan cinta", "ngabuburit": "menunggu berbuka puasa", "ngaku": "mengaku", "ngambil": "mengambil", "nganggur": "tidak punya pekerjaan", "ngapah": "kenapa", "ngaret": "terlambat", "ngasih": "memberikan", "ngebandel": "berbuat bandel", "ngegosip": "bergosip", "ngeklaim": "mengklaim", "ngeksis": "menjadi eksis", "ngeles": "berkilah", "ngelidur": "menggigau", "ngerampok": "merampok", "ngga": "tidak", "ngibul": "berbohong", "ngiler": "mau", "ngiri": "iri", "ngisiin": "mengisikan", "ngmng": "bicara", "ngomong": "bicara", "ngubek2": "mencari-cari", "ngurus": "mengurus", "nie": "ini", "nih": "ini", "niyh": "nih", "nmr": "nomor", "nntn": "nonton", "nobar": "nonton bareng", "np": "now playing", "ntar": "nanti", "ntn": "nonton", "numpuk": "bertumpuk", "nutupin": "menutupi", "nyari": "mencari", "nyekar": "menyekar", "nyicil": "mencicil", "nyoblos": "mencoblos", "nyokap": "ibu", "ogah": "tidak mau", "ol": "online", "ongkir": "ongkos kirim", "oot": "out of topic", "org2": "orang-orang", "ortu": "orang tua", "otda": "otonomi daerah", "otw": "on the way, sedang di jalan", "pacal": "pacar", "pake": "pakai", "pala": "kepala", "pansus": "panitia khusus", "parpol": "partai politik", "pasutri": "pasangan suami istri", "pd": "pada", "pede": "percaya diri", "pelatnas": "pemusatan latihan nasional", "pemda": "pemerintah daerah", "pemkot": "pemerintah kota", "pemred": "pemimpin redaksi", "penjas": "pendidikan jasmani", "perda": "peraturan daerah", "perhatiin": "perhatikan", "pesenan": "pesanan", "pgang": "pegang", "pi": "tapi", "pilkada": "pemilihan kepala daerah", "pisan": "sangat", "pk": "penjahat kelamin", "plg": "paling", "pmrnth": "pemerintah", "polantas": "polisi lalu lintas", "ponpes": "pondok pesantren", "pp": "pulang pergi", "prg": "pergi", "prnh": "pernah", "psen": "pesan", "pst": "pasti", "pswt": "pesawat", "pw": "posisi nyaman", "qmu": "kamu", "rakor": "rapat koordinasi", "ranmor": "kendaraan bermotor", "re": "reply", "ref": "referensi", "rehab": "rehabilitasi", "rempong": "sulit", "repp": "balas", "restik": "reserse narkotika", "rhs": "rahasia", "rmh": "rumah", "ru": "baru", "ruko": "rumah toko", "rusunawa": "rumah susun sewa", "ruz": "terus", "saia": "saya", "salting": "salah tingkah", "sampe": "sampai", "samsek": "sama sekali", "sapose": "siapa", "satpam": "satuan pengamanan", "sbb": "sebagai berikut", "sbh": "sebuah", "sbnrny": "sebenarnya", "scr": "secara", "sdgkn": "sedangkan", "sdkt": "sedikit", "se7": "setuju", "sebelas dua belas": "mirip", "sembako": "sembilan bahan pokok", "sempet": "sempat", "sendratari": "seni drama tari", "sgt": "sangat", "shg": "sehingga", "siech": "sih", "sikon": "situasi dan kondisi", "sinetron": "sinema elektronik", "siramin": "siramkan", "sj": "saja", "skalian": "sekalian", "sklh": "sekolah", "skt": "sakit", "slesai": "selesai", "sll": "selalu", "slma": "selama", "slsai": "selesai", "smpt": "sempat", "smw": "semua", "sndiri": "sendiri", "soljum": "sholat jumat", "songong": "sombong", "sory": "maaf", "sosek": "sosial-ekonomi", "sotoy": "sok tahu", "spa": "siapa", "sppa": "siapa", "spt": "seperti", "srtfkt": "sertifikat", "stiap": "setiap", "stlh": "setelah", "suk": "masuk", "sumpek": "sempit", "syg": "sayang", "t4": "tempat", "tajir": "kaya", "tau": "tahu", "taw": "tahu", "td": "tadi", "tdk": "tidak", "teh": "kakak perempuan", "telat": "terlambat", "telmi": "telat berpikir", "temen": "teman", "tengil": "menyebalkan", "tepar": "terkapar", "tggu": "tunggu", "tgu": "tunggu", "thankz": "terima kasih", "thn": "tahun", "tilang": "bukti pelanggaran", "tipiwan": "TvOne", "tks": "terima kasih", "tlp": "telepon", "tls": "tulis", "tmbah": "tambah", "tmen2": "teman-teman", "tmpah": "tumpah", "tmpt": "tempat", "tngu": "tunggu", "tnyta": "ternyata", "tokai": "tai", "toserba": "toko serba ada", "tpi": "tapi", "trdhulu": "terdahulu", "trima": "terima kasih", "trm": "terima", "trs": "terus", "trutama": "terutama", "ts": "penulis", "tst": "tahu sama tahu", "ttg": "tentang", "tuch": "tuh", "tuir": "tua", "tw": "tahu", "u": "kamu", "ud": "sudah", "udah": "sudah", "ujg": "ujung", "ul": "ulangan", "unyu": "lucu", "uplot": "unggah", "urang": "saya", "usah": "perlu", "utk": "untuk", "valas": "valuta asing", "w/": "dengan", "wadir": "wakil direktur", "wamil": "wajib militer", "warkop": "warung kopi", "warteg": "warung tegal", "wat": "buat", "wkt": "waktu", "wtf": "what the fuck", "xixixi": "tertawa", "ya": "iya", "yap": "iya", "yaudah": "ya sudah", "yawdah": "ya sudah", "yg": "yang", "yl": "yang lain", "yo": "iya", "yowes": "ya sudah", "yup": "iya", "7an": "tujuan", "ababil": "abg labil", "acc": "accord", "adlah": "adalah", "adoh": "aduh", "aha": "tertawa", "aing": "saya", "aja": "saja", "ajj": "saja", "aka": "dikenal juga sebagai", "akko": "aku", "akku": "aku", "akyu": "aku", "aljasa": "asal jadi saja", "ama": "sama", "ambl": "ambil", "anjir": "anjing", "ank": "anak", "ap": "apa", "apaan": "apa", "ape": "apa", "aplot": "unggah", "apva": "apa", "aqu": "aku", "asap": "sesegera mungkin", "aseek": "asyik", "asek": "asyik", "aseknya": "asyiknya", "asoy": "asyik", "astrojim": "astagfirullahaladzim", "ath": "kalau begitu", "atuh": "kalau begitu", "ava": "avatar", "aws": "awas", "ayang": "sayang", "ayok": "ayo", "bacot": "banyak bicara", "bales": "balas", "bangdes": "pembangunan desa", "bangkotan": "tua", "banpres": "bantuan presiden", "bansarkas": "bantuan sarana kesehatan", "bazis": "badan amal, zakat, infak, dan sedekah", "bcoz": "karena", "beb": "sayang", "bejibun": "banyak", "belom": "belum", "bener": "benar", "ber2": "berdua", "berdikari": "berdiri di atas kaki sendiri", "bet": "banget", "beti": "beda tipis", "beut": "banget", "bgd": "banget", "bgs": "bagus", "bhubu": "tidur", "bimbuluh": "bimbingan dan penyuluhan", "bisi": "kalau-kalau", "bkn": "bukan", "bl": "beli", "blg": "bilang", "blm": "belum", "bls": "balas", "bnchi": "benci", "bngung": "bingung", "bnyk": "banyak", "bohay": "badan aduhai", "bokep": "porno", "bokin": "pacar", "bole": "boleh", "bolot": "bodoh", "bonyok": "ayah ibu", "bpk": "bapak", "brb": "segera kembali", "brngkt": "berangkat", "brp": "berapa", "brur": "saudara laki-laki", "bsa": "bisa", "bsk": "besok", "bu_bu": "tidur", "bubarin": "bubarkan", "buber": "buka bersama", "bujubune": "luar biasa", "buser": "buru sergap", "bwhn": "bawahan", "byar": "bayar", "byr": "bayar", "c8": "chat", "cabut": "pergi", "caem": "cakep", "cama-cama": "sama-sama", "cangcut": "celana dalam", "cape": "capek", "caur": "jelek", "cekak": "tidak ada uang", "cekidot": "coba lihat", "cemplungin": "cemplungkan", "ceper": "pendek", "ceu": "kakak perempuan", "cewe": "cewek", "cibuk": "sibuk", "cin": "cinta", "ciye": "cie", "ckck": "ck", "clbk": "cinta lama bersemi kembali", "cmpr": "campur", "cnenk": "senang", "congor": "mulut", "cow": "cowok", "coz": "karena", "cpa": "siapa", "gokil": "gila", "gombal": "suka merayu", "gpl": "tidak pakai lama", "gpp": "tidak apa-apa", "gretong": "gratis", "gt": "begitu", "gtw": "tidak tahu", "gue": "saya", "guys": "teman-teman", "gws": "cepat sembuh", "haghaghag": "tertawa", "hakhak": "tertawa", "handak": "bahan peledak", "hansip": "pertahanan sipil", "hellow": "halo", "helow": "halo", "hi": "hai", "hlng": "hilang", "hnya": "hanya", "houm": "rumah", "hrs": "harus", "hubad": "hubungan angkatan darat", "hubla": "perhubungan laut", "huft": "mengeluh", "humas": "hubungan masyarakat", "idk": "saya tidak tahu", "ilfeel": "tidak suka", "imba": "jago sekali", "imoet": "imut", "info": "informasi", "itung": "hitung", "isengin": "bercanda", "iyala": "iya lah", "iyo": "iya", "jablay": "jarang dibelai", "jadul": "jaman dulu", "jancuk": "anjing", "jd": "jadi", "jdikan": "jadikan", "jg": "juga", "jgn": "jangan", "jijay": "jijik", "jkt": "jakarta", "jnj": "janji", "jth": "jatuh", "jurdil": "jujur adil", "jwb": "jawab", "ka": "kakak", "kabag": "kepala bagian", "kacian": "kasihan", "kadit": "kepala direktorat", "kaga": "tidak", "kaka": "kakak", "kamtib": "keamanan dan ketertiban", "kamuh": "kamu", "kamyu": "kamu", "kapt": "kapten", "kasat": "kepala satuan", "kasubbid": "kepala subbidang", "kau": "kamu", "kbar": "kabar", "kcian": "kasihan", "keburu": "terlanjur", "kedubes": "kedutaan besar", "kek": "seperti", "keknya": "kayaknya", "keliatan": "kelihatan", "keneh": "masih", "kepikiran": "terpikirkan", "kepo": "mau tahu urusan orang", "kere": "tidak punya uang", "kesian": "kasihan", "ketauan": "ketahuan", "keukeuh": "keras kepala", "khan": "kan", "kibus": "kaki busuk", "kk": "kakak", "klian": "kalian", "klo": "kalau", "kluarga": "keluarga", "klwrga": "keluarga", "kmari": "kemari", "kmpus": "kampus", "kn": "kan", "knl": "kenal", "knpa": "kenapa", "kog": "kok", "kompi": "komputer", "komtiong": "komunis Tiongkok", "konjen": "konsulat jenderal", "koq": "kok", "kpd": "kepada",
        "kptsan":"keputusan", "krik": "garing", "krn": "karena", "ktauan": "ketahuan", "ktny": "katanya", "kudu": "harus", "kuq": "kok", "ky": "seperti", "kykny": "kayanya", "laka": "kecelakaan",
        "lambreta": "lambat", "lansia": "lanjut usia", "lapas": "lembaga pemasyarakatan", "lbur": "libur", "lekong": "laki-laki", "lg": "lagi", "lgkp": "lengkap", "lht": "lihat",
        "linmas": "perlindungan masyarakat", "lmyan": "lumayan", "lngkp": "lengkap", "loch": "loh", "lol": "tertawa", "lom": "belum", "loupz": "cinta", "lowh": "kamu", "lu": "kamu",
        "luchu": "lucu", "luff": "cinta", "luph": "cinta", "lw": "kamu", "lwt": "lewat", "maaciw": "terima kasih", "mabes": "markas besar", "macem-macem": "macam-macam", "madesu": "masa depan suram",
        "maen": "main", "mahatma": "maju sehat bersama", "mak": "ibu", "makasih": "terima kasih", "malah": "bahkan", "malu2in": "memalukan", "mamz": "makan", "manies": "manis", "mantep": "mantap",
        "markus": "makelar kasus", "mba": "mbak", "mending": "lebih baik", "mgkn": "mungkin", "mhn": "mohon", "miker": "minuman keras", "milis": "mailing list", "mksd": "maksud", "mls": "malas",
        "mnt": "minta", "moge": "motor gede", "mokat": "mati", "mosok": "masa", "msh": "masih", "mskpn": "meskipun", "msng2": "masing-masing", "muahal": "mahal", "muker": "musyawarah kerja",
        "mumet": "pusing", "muna": "munafik", "munaslub": "musyawarah nasional luar biasa", "musda": "musyawarah daerah", "muup": "maaf", "muuv": "maaf", "nal": "kenal", "nangis": "menangis",
        "naon": "apa", "napol": "narapidana politik", "naq": "anak", "narsis": "bangga pada diri sendiri", "nax": "anak", "ndak": "tidak", "ndut": "gendut", "nekolim": "neokolonialisme",
        "nelfon": "menelepon", "ngabis2in": "menghabiskan", "ngakak": "tertawa", "ngambek": "marah", "ngampus": "pergi ke kampus", "ngantri": "mengantri", "ngapain": "sedang apa",
        "ngaruh": "berpengaruh", "ngawur": "berbicara sembarangan", "ngeceng": "kumpul bareng-bareng", "ngeh": "sadar", "ngekos": "tinggal di kos", "ngelamar": "melamar", "ngeliat": "melihat",
        "ngemeng": "bicara terus-terusan", "ngerti": "mengerti", "nggak": "tidak", "ngikut": "ikut", "nginep": "menginap", "ngisi": "mengisi", "ngmg": "bicara", "ngocol": "lucu",
        "ngomongin": "membicarakan", "ngumpul": "berkumpul", "ni": "ini", "nyasar": "tersesat", "nyariin": "mencari", "nyiapin": "mempersiapkan", "nyiram": "menyiram", "nyok": "ayo", "o/": "oleh",
        "ok": "ok", "priksa": "periksa", "pro": "profesional", "psn": "pesan", "psti": "pasti", "puanas": "panas", "qmo": "kamu", "qt": "kita", "rame": "ramai", "raskin": "rakyat miskin",
        "red": "redaksi", "reg": "register", "rejeki": "rezeki", "renstra": "rencana strategis", "reskrim": "reserse kriminal", "sni": "sini", "somse": "sombong sekali", "sorry": "maaf",
        "sosbud": "sosial-budaya", "sospol": "sosial-politik", "sowry": "maaf", "spd": "sepeda", "sprti": "seperti", "spy": "supaya", "stelah": "setelah", "subbag": "subbagian",
        "sumbangin": "sumbangkan", "sy": "saya", "syp": "siapa", "tabanas": "tabungan pembangunan nasional", "tar": "nanti", "taun": "tahun", "tawh": "tahu", "tdi": "tadi", "te2p": "tetap",
        "tekor": "rugi", "telkom": "telekomunikasi", "telp": "telepon", "temen2": "teman-teman", "tengok": "menjenguk", "terbitin": "terbitkan", "tgl": "tanggal", "thanks": "terima kasih",
        "thd": "terhadap", "thx": "terima kasih", "tipi": "TV", "tkg": "tukang", "tll": "terlalu", "tlpn": "telepon", "tman": "teman", "tmbh": "tambah", "tmn2": "teman-teman", "tmph": "tumpah",
        "tnda": "tanda", "tnh": "tanah", "togel": "toto gelap", "tp": "tapi", "tq": "terima kasih", "trgntg": "tergantung", "trims": "terima kasih", "cb": "coba", "y": "ya", "munfik": "munafik",
        "reklamuk": "reklamasi", "sma": "sama", "tren": "trend", "ngehe": "kesal", "mz": "mas", "analisise": "analisis", "sadaar": "sadar", "sept": "september", "nmenarik": "menarik",
        "zonk": "bodoh", "rights": "benar", "simiskin": "miskin", "ngumpet": "sembunyi", "hardcore": "keras", "akhirx": "akhirnya", "solve": "solusi", "watuk": "batuk", "ngebully": "intimidasi",
        "masy": "masyarakat", "still": "masih", "tauk": "tahu", "mbual": "bual", "tioghoa": "tionghoa", "ngentotin": "senggama", "kentot": "senggama", "faktakta": "fakta", "sohib": "teman",
        "rubahnn": "rubah", "trlalu": "terlalu", "nyela": "cela", "heters": "pembenci", "nyembah": "sembah", "most": "paling", "ikon": "lambang", "light": "terang", "pndukung": "pendukung",
        "setting": "atur", "seting": "akting", "next": "lanjut", "waspadalah": "waspada", "gantengsaya": "ganteng", "parte": "partai", "nyerang": "serang", "nipu": "tipu", "ktipu": "tipu",
        "jentelmen": "berani", "buangbuang": "buang", "tsangka": "tersangka", "kurng": "kurang", "ista": "nista", "less": "kurang", "koar": "teriak", "paranoid": "takut", "problem": "masalah",
        "tahi": "kotoran", "tirani": "tiran", "tilep": "tilap", "happy": "bahagia", "tak": "tidak", "penertiban": "tertib", "uasai": "kuasa", "mnolak": "tolak", "trending": "trend",
        "taik": "tahi", "wkwkkw": "tertawa", "ahokncc": "ahok", "istaa": "nista", "benarjujur": "jujur", "mgkin": "mungkin", 'ga':'tidak', 'cwe':'perempuan', 'dl':'dulu', 'rumayan':'lumayan',
        'ny':'nya', 'htm':'harga tiket masuk', 'cm':'cuma', 'slalu':'selalu', 'tingi':'tinggi','neng':'senang'}
processed_comments = []

for sentence in data['text']:
    # Remove all the special characters
    processed_comment = re.sub(r'\W', ' ', str(sentence))

    # Converting to Lowercase
    processed_comment = processed_comment.lower()

    #Remove number
    processed_comment = re.sub(r'\d+', ' ', processed_comment)

    # remove all single characters
    processed_comment = re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_comment)

    #remove duplicate character
    pattern=reg.compile(r"(.)\1{1,}",reg.DOTALL)
    processed_comment=pattern.sub(r"\1",processed_comment)


    #Corrected Slang words
    words = processed_comment.split()
    rfrm=[slangs[word] if word in slangs else word for word in words]
    processed_comment= " ".join(rfrm)

    #remove stopword
    factory = StopWordRemoverFactory()
    more_stopword = ['tak', 'jd', 'per', 'nya', 'terjemah', 'diterjemahkan', 'oleh', 'gogle', 'google' ,'nan', 'baik', 'sangat', 'batas', 'coba',
                        'ada','bersih', 'salur', 'baru', 'purwokerto', 'batas', 'hotel', 'coba', 'putus', 'ada','sama', 'suka', 'bilang',
                        'com', 'kamu', 'http', 'https', 'htps', 'htp', 'gak', 'jadi', 'lebih', 'kalau', 'banyak', 'jangan', 'iya', 'kok',
                        'apa', 'paling', 'semua', 'lah', 'ihwm', 'od', 'pakai', 'hayo', 'no', 'ihwjm', 'od'] #menambahkan stopword
    stopwords = factory.get_stop_words() + more_stopword
    temp = [t for t in re.findall(r'\b[a-z]+-?[a-z]+\b',processed_comment) if t not in stopwords]
    processed_comment = ' '.join(temp)

    #stemming
    stemmer = StemmerFactory().create_stemmer()
    processed_comment = stemmer.stem(processed_comment)

    # Substituting multiple spaces with single space
    processed_comment = re.sub(r'\s+', ' ', processed_comment, flags=re.I)

    processed_comments.append(processed_comment)

#output data Preprocessing
processed_comments

['harap politis gaya lacur pramagtis jelas warna politik punya prinsip politik milik integritas cukup yakin politis model begini suka rakyat',
 'syahganda gugat syahganda mg benar mg lacur intlektual syahganda buka bukan propaganda hasil survei deny depan bawaslu politik rmol co read syahganda buka bukan propaganda hasil survei deny depan bawaslu',
 'pdemokrat org punya akal sehat maju bangsa beberapa orang begundal serta lacur politik grasak grusuk salah satu heran twiter ferdinandhaean status',
 'cypridophobia fobia takut lacur tular sakit kelamin',
 'ramai kerja waktu malam biasa lacur siapa idea suh kerja malam tu takfahamtakapa',
 'lacur intlektual kpucurangrakyatmelawan kpungejarquickcounthtps twiter suwandaben status',
 'lembaga lacur lsi denyja germo rupa kejar target atas propinsi opini publik simultan dg operasi ampo suara mana hina betul hidup merekahtps twiter status',
 'bangun gada orang sukses kasur lacur selfreminder',
 'abai lacur agama hidup nkri hidup presiden joko wi

In [6]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
processed_comments = ''.join(processed_comments)

In [9]:
type(processed_comments)

str

In [10]:
from nltk.tokenize import word_tokenize
Docs = processed_comments
Sentences = nltk.sent_tokenize(Docs)
print ("Kalimat Awal: ", Sentences)
hasil_tokenizing = nltk.word_tokenize(Docs)
print("Setelah Tokenizing: ", hasil_tokenizing)

Kalimat Awal:  ['harap politis gaya lacur pramagtis jelas warna politik punya prinsip politik milik integritas cukup yakin politis model begini suka rakyatsyahganda gugat syahganda mg benar mg lacur intlektual syahganda buka bukan propaganda hasil survei deny depan bawaslu politik rmol co read syahganda buka bukan propaganda hasil survei deny depan bawaslupdemokrat org punya akal sehat maju bangsa beberapa orang begundal serta lacur politik grasak grusuk salah satu heran twiter ferdinandhaean statuscypridophobia fobia takut lacur tular sakit kelaminramai kerja waktu malam biasa lacur siapa idea suh kerja malam tu takfahamtakapalacur intlektual kpucurangrakyatmelawan kpungejarquickcounthtps twiter suwandaben statuslembaga lacur lsi denyja germo rupa kejar target atas propinsi opini publik simultan dg operasi ampo suara mana hina betul hidup merekahtps twiter statusbangun gada orang sukses kasur lacur selfreminderabai lacur agama hidup nkri hidup presiden joko widodomungkin sat bersih se

In [11]:
len(hasil_tokenizing)

4553

In [12]:
#Ekstraksi fitur / representasi dokumen menggunakan TF
from sklearn.feature_extraction.text import CountVectorizer
tf_vectorizer = CountVectorizer(max_df=1.0, min_df=1)

tf = tf_vectorizer.fit_transform(hasil_tokenizing)

#hasil representasi
tf_terms = tf_vectorizer.get_feature_names_out()
print(tf_vectorizer.get_feature_names_out())
matrix = tf.toarray()
print(matrix)

['abal' 'abdu' 'abng' ... 'zina' 'zonamerahyesus' 'zul']
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


### **Menggunakan LDA**

In [14]:
from sklearn.decomposition import LatentDirichletAllocation as LDA

n_topics = 10 #untuk mendapatkan jumlah topik terbaik perlu trial
lda = LDA(n_components=n_topics, learning_method='batch', random_state=0)
lda.fit(tf)
lda

LatentDirichletAllocation(random_state=0)

In [17]:
vsm_topics = lda.transform(tf)
#tampilkan hasil
print(vsm_topics)

[[0.05 0.05 0.05 ... 0.05 0.05 0.05]
 [0.05 0.55 0.05 ... 0.05 0.05 0.05]
 [0.05 0.05 0.05 ... 0.05 0.05 0.05]
 ...
 [0.05 0.05 0.55 ... 0.05 0.05 0.05]
 [0.05 0.05 0.05 ... 0.05 0.55 0.05]
 [0.05 0.05 0.55 ... 0.05 0.05 0.05]]


In [27]:
len(vsm_topics)

4553

In [20]:
#Tampilkan nilai-nilai setiap fitur
print(lda.components_)

[[1.09999998 0.1        0.1        ... 0.1        0.1        0.1       ]
 [0.1        0.1        1.09999998 ... 3.09999999 0.1        0.1       ]
 [0.1        0.1        0.1        ... 0.1        0.1        0.1       ]
 ...
 [0.1        5.09999999 0.1        ... 0.1        0.1        3.09999999]
 [0.1        0.1        0.1        ... 0.1        0.1        0.1       ]
 [0.1        0.1        0.1        ... 0.1        0.1        0.1       ]]


In [21]:
#hasil label topic model untuk setiap dokumen
import numpy as np
topics = np.argmax(vsm_topics, axis=1)
topics

array([5, 1, 3, ..., 2, 8, 2])

In [28]:
#mencetak word fitur dengan nilai tertinggi pada setiap topik
n_top_words = 10 # jumlah fitur tertinggi yang kita tentukan
topic_words = {}
for topic, comp in enumerate(lda.components_):
 word_idx = np.argsort(comp)[::-1][:n_top_words] # susun indeks secara terbalik, sehingga yang paling tinggi muncul di awal.
 # store the words most relevant to the topic
 topic_words[topic] = [tf_vectorizer.get_feature_names_out()[i]+' '+str(comp[i]) for i in word_idx]

In [29]:
type(topic_words)

dict

In [30]:
topic_words

{0: ['lacur 232.09999998665333',
  'tutup 11.099999986178311',
  'negara 8.099999985984805',
  'rakyat 8.099999985984805',
  'uang 8.099999985984805',
  'sebut 7.099999985881927',
  'hehamahua 7.099999985881927',
  'dg 6.099999985743121',
  'salah 6.099999985743121',
  'statuslacur 5.099999985545637'],
 1: ['ahok 14.099999987493604',
  'cina 12.099999987433058',
  'bikin 12.099999987433055',
  'teman 9.099999987290147',
  'bangsa 9.099999987290145',
  'seksual 9.099999987290143',
  'islam 7.099999987123999',
  'kontol 7.099999987123998',
  'mk 7.099999987123998',
  'macam 6.099999986997406'],
 2: ['ras 57.09999998695473',
  'twiter 55.09999998695142',
  'verbal 23.09999998681797',
  'punya 16.099999986716277',
  'dasar 13.09999998663843',
  'mana 11.099999986562308',
  'biasa 10.099999986512502',
  'kerja 10.099999986512502',
  'jual 10.099999986512502',
  'tuhan 10.099999986512502'],
 3: ['jadi 23.099999986880732',
  'mau 21.099999986858805',
  'kata 14.09999998673166',
  'anak 13.099

In [32]:
for topic, words in topic_words.items():
 print('Topic: %d' % topic)
 print(' %s' % ', '.join(words))

Topic: 0
 lacur 232.09999998665333, tutup 11.099999986178311, negara 8.099999985984805, rakyat 8.099999985984805, uang 8.099999985984805, sebut 7.099999985881927, hehamahua 7.099999985881927, dg 6.099999985743121, salah 6.099999985743121, statuslacur 5.099999985545637
Topic: 1
 ahok 14.099999987493604, cina 12.099999987433058, bikin 12.099999987433055, teman 9.099999987290147, bangsa 9.099999987290145, seksual 9.099999987290143, islam 7.099999987123999, kontol 7.099999987123998, mk 7.099999987123998, macam 6.099999986997406
Topic: 2
 ras 57.09999998695473, twiter 55.09999998695142, verbal 23.09999998681797, punya 16.099999986716277, dasar 13.09999998663843, mana 11.099999986562308, biasa 10.099999986512502, kerja 10.099999986512502, jual 10.099999986512502, tuhan 10.099999986512502
Topic: 3
 jadi 23.099999986880732, mau 21.099999986858805, kata 14.09999998673166, anak 13.099999986702063, prabowo 11.09999998662631, lihat 11.099999986626308, presiden 10.099999986576739, banget 10.0999999

### **Menggunakan LSA**

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer
vektor = TfidfVectorizer(max_features=400)

In [44]:
df = pd.read_csv('Kasus1 (2).csv', delimiter=';')

In [45]:
df.head()

,screen_name,username,user_id,tweet_id,tweet_url,timestamp,timestamp_epochs,text,mengandung pelecehan,text_html,...,has_media,img_urls,video_url,likes,retweets,replies,is_replied,is_reply_to,parent_tweet_id,reply_to_users
0,BimaBurhanuddin,Burhanuddin Bima,3058961791,"1,12E+18",/BimaBurhanuddin/status/1124100611925000193,02/05/2019 23:57,1556841445,Apa yg bisa diharapkan dari politisi gaya pela...,1,"<p class=""TweetTextSize js-tweet-text tweet-te...",...,False,[],NaN,0,0,0,False,False,NaN,[]
1,ngowatchindo,Waw Lam'alif,"8,95E+17","1,12E+18",/ngowatchindo/status/1124099972046163968,02/05/2019 23:54,1556841292,"Kalau sy jadi ⁦@DennyJA_WORLD⁩ , Syahganda aka...",1,"<p class=""TweetTextSize js-tweet-text tweet-te...",...,False,[],NaN,1,1,0,False,False,NaN,[]
2,socmed_anak,AnakSocmed,174945628,"1,12E+18",/socmed_anak/status/1124098250083131393,02/05/2019 23:48,1556840882,Di @PDemokrat banyak org2 yg punya akal sehat ...,1,"<p class=""TweetTextSize js-tweet-text tweet-te...",...,False,[],NaN,0,0,0,False,False,NaN,[]
3,PelajaranOnline,PelajaranOnline,589466301,"1,12E+18",/PelajaranOnline/status/1124087714154975232,02/05/2019 23:06,1556838370,Cypridophobia adalah fobia atau takut pada pel...,0,"<p class=""TweetTextSize js-tweet-text tweet-te...",...,False,[],NaN,0,0,0,False,False,NaN,[]
4,Lee_Botak,Lee Botak,509046818,"1,12E+18",/Lee_Botak/status/1124084310640869379,02/05/2019 22:52,1556837558,Yang RAMAI kerja waktu malam ni biasanya PELAC...,1,"<p class=""TweetTextSize js-tweet-text tweet-te...",...,False,[],NaN,0,0,0,False,False,NaN,[]


In [46]:
df = df['text']

In [48]:
df = pd.DataFrame(df)

In [49]:
df.head()

,text
0,Apa yg bisa diharapkan dari politisi gaya pela...
1,"Kalau sy jadi ⁦@DennyJA_WORLD⁩ , Syahganda aka..."
2,Di @PDemokrat banyak org2 yg punya akal sehat ...
3,Cypridophobia adalah fobia atau takut pada pel...
4,Yang RAMAI kerja waktu malam ni biasanya PELAC...


In [50]:
#menghitung tf-idf dengan TfidfTransformer
vektor_dt=vektor.fit_transform(df['text'].values.astype('U'))
print (vektor_dt)
print (vektor_dt.shape)

  (0, 314)	0.19163539454172504
  (0, 8)	0.17789102930169112
  (0, 327)	0.18418784376347547
  (0, 68)	0.21994937405570378
  (0, 360)	0.36837568752695093
  (0, 320)	0.1877421225871695
  (0, 322)	0.16132841303982653
  (0, 46)	0.22906443953931876
  (0, 100)	0.20620500881566986
  (0, 312)	0.16996005941596481
  (0, 390)	0.22906443953931876
  (0, 170)	0.20075046002534003
  (0, 368)	0.30212522247949253
  (0, 293)	0.05508471193137627
  (0, 304)	0.4124100176313397
  (0, 76)	0.14412250099738255
  (0, 94)	0.22906443953931876
  (0, 56)	0.16132841303982653
  (0, 397)	0.21136960269893074
  (0, 15)	0.16541666597476792
  (1, 0)	0.10885684061020809
  (1, 303)	0.06915932608129484
  (1, 150)	0.06764600560780328
  (1, 35)	0.24672209448913082
  (1, 85)	0.21771368122041618
  :	:
  (396, 293)	0.10587080090938653
  (396, 76)	0.2769981738066525
  (396, 397)	0.20312232143024225
  (397, 167)	0.47146329323659725
  (397, 205)	0.5923139922522108
  (397, 265)	0.5276556682391096
  (397, 166)	0.3556135938846105
  (397,

In [62]:
# topic modeling using LSA
from sklearn.decomposition import TruncatedSVD
lsa_model = TruncatedSVD(n_components=5, n_iter=10, random_state=42)

lsa_top=lsa_model.fit_transform(vektor_dt)
print(lsa_top)

[[ 0.17744506  0.08874096 -0.10785159 -0.02944705  0.02728072]
 [ 0.11154731  0.02149548 -0.00873574 -0.01155643  0.05291242]
 [ 0.39835678  0.26178951  0.18893442  0.01301177  0.15256777]
 ...
 [ 0.17377837 -0.03168128 -0.08691269 -0.04158952 -0.05964318]
 [ 0.2306375  -0.04229336 -0.08620943 -0.0686838  -0.05236789]
 [ 0.30541248  0.21738289  0.0008827  -0.05408745 -0.02098137]]


In [63]:
len(lsa_top)

400

In [53]:
lsa_model.get_params()

{'algorithm': 'randomized',
 'n_components': 10,
 'n_iter': 10,
 'n_oversamples': 10,
 'power_iteration_normalizer': 'auto',
 'random_state': 42,
 'tol': 0.0}

In [64]:
# Memunculkan nilai lsa setiap topik
l=lsa_top[0]
print("Topik- Topik:")
for i,topic in enumerate(l):
  print("Topic ",i," : ",topic*100)

Topik- Topik:
Topic  0  :  17.744505865926005
Topic  1  :  8.874095569744172
Topic  2  :  -10.785158550673081
Topic  3  :  -2.9447045260575107
Topic  4  :  2.7280717794039826


In [65]:
# Memunculkan jumlah kata-kata dalam setiap topik
print(lsa_model.components_.shape)
print(lsa_model.components_)

(5, 400)
[[ 0.01237606  0.01656476  0.06060376 ...  0.1490447   0.01749481
   0.02267373]
 [ 0.0071464   0.00651101  0.06882746 ...  0.156516   -0.01511378
  -0.01946296]
 [ 0.01302258 -0.01771778 -0.06452446 ... -0.08535616  0.00861879
   0.00948279]
 [-0.00106424 -0.00627941 -0.01326746 ...  0.0050432   0.02940468
   0.0363545 ]
 [ 0.00580298  0.00591098  0.03400822 ...  0.13602107  0.04926822
   0.06390896]]


In [66]:
# Word/ kata paling penting dalam setiap topik
vocab = vektor.get_feature_names_out()
for i, comp in enumerate(lsa_model.components_):
    vocab_comp = zip(vocab, comp)
    sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:10]
    print("Topic "+str(i)+": ")
    for t in sorted_words:
        print(t[0],end=", ")
    print("\n")

Topic 0: 
pelacur, politik, com, twitter, yg, di, status, ini, dan, https, 

Topic 1: 
com, twitter, rasis, status, https, mesum, yg, itu, ini, dan, 

Topic 2: 
twitter, com, status, https, politik, intlektual, pic, dibilang, tutup01tusuk02, disebut, 

Topic 3: 
mesum, politik, aja, otak, lo, juga, gak, babi, ini, yusrilnorakdankasarmainnya, 

Topic 4: 
politik, rasis, yg, aja, para, cina, ya, lu, yusrilnorakdankasarmainnya, ini, 



In [61]:
for topic, words in topic_words.items():
 print('Topic: %d' % topic)
 print(' %s' % ', '.join(words))

Topic: 0
 lacur 232.09999998665333, tutup 11.099999986178311, negara 8.099999985984805, rakyat 8.099999985984805, uang 8.099999985984805, sebut 7.099999985881927, hehamahua 7.099999985881927, dg 6.099999985743121, salah 6.099999985743121, statuslacur 5.099999985545637
Topic: 1
 ahok 14.099999987493604, cina 12.099999987433058, bikin 12.099999987433055, teman 9.099999987290147, bangsa 9.099999987290145, seksual 9.099999987290143, islam 7.099999987123999, kontol 7.099999987123998, mk 7.099999987123998, macam 6.099999986997406
Topic: 2
 ras 57.09999998695473, twiter 55.09999998695142, verbal 23.09999998681797, punya 16.099999986716277, dasar 13.09999998663843, mana 11.099999986562308, biasa 10.099999986512502, kerja 10.099999986512502, jual 10.099999986512502, tuhan 10.099999986512502
Topic: 3
 jadi 23.099999986880732, mau 21.099999986858805, kata 14.09999998673166, anak 13.099999986702063, prabowo 11.09999998662631, lihat 11.099999986626308, presiden 10.099999986576739, banget 10.0999999

## Tugas Praktikum
1. Cari dataset teks untuk dilakukan topik modeling
2. Buat topik modeling menggunakan LDA & LSA (jangan lupa dibersihkan teksnya)
3. Berikan kesimpulan dari hasil uji coba kamu
4. Kumpulkan di LMS dalam bentuk PDF (ipynbnya di pdf in)